In [70]:
import os
import csv
import pandas as pd 
from sqlalchemy import create_engine
import pymysql
pymysql.install_as_MySQLdb()

In [22]:
# Naming Ranking to convert to PD 
ranking_history = "./data/rankings_history.csv"
ufc_fighters = "./data/ufcdata.csv"

In [23]:
# Use Pandas to read data
ranking_df = pd.read_csv(ranking_history)
ufc_fighters_df = pd.read_csv(ufc_fighters)

#data_file_pd
ranking_df.head()
ufc_fighters_df.head()

,BPrev,BStreak,B_Age,B_Height,B_HomeTown,B_ID,B_Location,B_Name,B_Weight,B__Round1_Grappling_Reversals_Landed,...,R__Round5_TIP_Ground Time,R__Round5_TIP_Guard Control Time,R__Round5_TIP_Half Guard Control Time,R__Round5_TIP_Misc. Ground Control Time,R__Round5_TIP_Mount Control Time,R__Round5_TIP_Neutral Time,R__Round5_TIP_Side Control Time,R__Round5_TIP_Standing Time,winby,winner
0,0,0,38.0,193.0,Hounslow England,808,Amsterdam The Netherlands,Alistair Overeem,120.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DEC,blue
1,0,0,36.0,172.0,"Chicago, Illinois United States",1054,"Chicago, Illinois United States",Ricardo Lamas,65.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DEC,red
2,0,0,39.0,167.0,"Isla Vista , California USA",959,"Sacramento, California USA",Urijah Faber,61.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,KO/TKO,red
3,0,0,33.0,167.0,"San Diego, CA USA",1056,"San Diego, CA USA",Danny Martinez,56.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DEC,red
4,0,0,36.0,185.0,Southampton England,2005,Southampton England,Tom Watson,84.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DEC,red


In [91]:
new_ranking_df = ranking_df[["date", "fighter", "weightclass", "rank"]]
new_ranking_df.head()

,date,fighter,weightclass,rank
0,2/4/13,Anderson Silva,Pound-for-Pound,1
1,2/4/13,Jon Jones,Pound-for-Pound,2
2,2/4/13,Georges St-Pierre,Pound-for-Pound,3
3,2/4/13,Jose Aldo,Pound-for-Pound,4
4,2/4/13,Benson Henderson,Pound-for-Pound,5


In [37]:
new_ranking_df = new_ranking_df.rename(columns={"date": "Date", "fighter": "Fighter", "weightclass": "WeightClass", "rank": "Rank"})
new_ranking_df.head()

,Date,Fighter,WeightClass,Rank
0,2/4/13,Anderson Silva,Pound-for-Pound,1
1,2/4/13,Jon Jones,Pound-for-Pound,2
2,2/4/13,Georges St-Pierre,Pound-for-Pound,3
3,2/4/13,Jose Aldo,Pound-for-Pound,4
4,2/4/13,Benson Henderson,Pound-for-Pound,5


In [101]:
new_ufc_fighters_df = ufc_fighters_df[[ "Event_ID","Fight_ID","Date",'B_ID','B_Name', 'B_HomeTown', 'B_Location','B_Age', 'B_Weight',
                                       'R_ID','R_Name', 'R_HomeTown', 'R_Location','R_Age', 'R_Weight','winner', 'winby']].copy()


new_ufc_fighters_df.head()

,Event_ID,Fight_ID,Date,B_ID,B_Name,B_HomeTown,B_Location,B_Age,B_Weight,R_ID,R_Name,R_HomeTown,R_Location,R_Age,R_Weight,winner,winby
0,646,4580,2/3/14,808,Alistair Overeem,Hounslow England,Amsterdam The Netherlands,38.0,120.0,377,Frank Mir,"Las Vegas, Nevada USA","Las Vegas, Nevada USA",39.0,119.0,blue,DEC
1,646,4589,2/3/14,1054,Ricardo Lamas,"Chicago, Illinois United States","Chicago, Illinois United States",36.0,65.0,1052,Jose Aldo,Manaus Brazil,Rio de Janeiro Brazil,32.0,65.0,red,DEC
2,646,4590,2/3/14,959,Urijah Faber,"Isla Vista , California USA","Sacramento, California USA",39.0,61.0,1527,Renan Barao,Natal Brazil,Rio de Janeiro Brazil,31.0,61.0,red,KO/TKO
3,646,4605,2/3/14,1056,Danny Martinez,"San Diego, CA USA","San Diego, CA USA",33.0,56.0,1253,Chris Cariaso,"San Jose, California USA","Tucson, Arizona USA",37.0,56.0,red,DEC
4,646,4631,2/3/14,2005,Tom Watson,Southampton England,Southampton England,36.0,84.0,464,Nick Catone,"Englewood, NJ USA","Brick, NJ USA",37.0,84.0,red,DEC


In [102]:
new_ufc_fighters_df = new_ufc_fighters_df.rename(columns={'winner': "WinningColor", 'winby': 'Winby'})

new_ufc_fighters_df.head()

,Event_ID,Fight_ID,Date,B_ID,B_Name,B_HomeTown,B_Location,B_Age,B_Weight,R_ID,R_Name,R_HomeTown,R_Location,R_Age,R_Weight,WinningColor,Winby
0,646,4580,2/3/14,808,Alistair Overeem,Hounslow England,Amsterdam The Netherlands,38.0,120.0,377,Frank Mir,"Las Vegas, Nevada USA","Las Vegas, Nevada USA",39.0,119.0,blue,DEC
1,646,4589,2/3/14,1054,Ricardo Lamas,"Chicago, Illinois United States","Chicago, Illinois United States",36.0,65.0,1052,Jose Aldo,Manaus Brazil,Rio de Janeiro Brazil,32.0,65.0,red,DEC
2,646,4590,2/3/14,959,Urijah Faber,"Isla Vista , California USA","Sacramento, California USA",39.0,61.0,1527,Renan Barao,Natal Brazil,Rio de Janeiro Brazil,31.0,61.0,red,KO/TKO
3,646,4605,2/3/14,1056,Danny Martinez,"San Diego, CA USA","San Diego, CA USA",33.0,56.0,1253,Chris Cariaso,"San Jose, California USA","Tucson, Arizona USA",37.0,56.0,red,DEC
4,646,4631,2/3/14,2005,Tom Watson,Southampton England,Southampton England,36.0,84.0,464,Nick Catone,"Englewood, NJ USA","Brick, NJ USA",37.0,84.0,red,DEC


In [103]:
# Reference multiple columns within a DataFrame
new_ufc_fighters_df[["Event_ID","Fight_ID", "B_Name", "R_Name", "Date", "WinningColor", "Winby"]].head()

,Event_ID,Fight_ID,B_Name,R_Name,Date,WinningColor,Winby
0,646,4580,Alistair Overeem,Frank Mir,2/3/14,blue,DEC
1,646,4589,Ricardo Lamas,Jose Aldo,2/3/14,red,DEC
2,646,4590,Urijah Faber,Renan Barao,2/3/14,red,KO/TKO
3,646,4605,Danny Martinez,Chris Cariaso,2/3/14,red,DEC
4,646,4631,Tom Watson,Nick Catone,2/3/14,red,DEC


In [104]:
# Updated and named DFs 
new_ranking_df.head()
new_ufc_fighters_df.head()

,Event_ID,Fight_ID,Date,B_ID,B_Name,B_HomeTown,B_Location,B_Age,B_Weight,R_ID,R_Name,R_HomeTown,R_Location,R_Age,R_Weight,WinningColor,Winby
0,646,4580,2/3/14,808,Alistair Overeem,Hounslow England,Amsterdam The Netherlands,38.0,120.0,377,Frank Mir,"Las Vegas, Nevada USA","Las Vegas, Nevada USA",39.0,119.0,blue,DEC
1,646,4589,2/3/14,1054,Ricardo Lamas,"Chicago, Illinois United States","Chicago, Illinois United States",36.0,65.0,1052,Jose Aldo,Manaus Brazil,Rio de Janeiro Brazil,32.0,65.0,red,DEC
2,646,4590,2/3/14,959,Urijah Faber,"Isla Vista , California USA","Sacramento, California USA",39.0,61.0,1527,Renan Barao,Natal Brazil,Rio de Janeiro Brazil,31.0,61.0,red,KO/TKO
3,646,4605,2/3/14,1056,Danny Martinez,"San Diego, CA USA","San Diego, CA USA",33.0,56.0,1253,Chris Cariaso,"San Jose, California USA","Tucson, Arizona USA",37.0,56.0,red,DEC
4,646,4631,2/3/14,2005,Tom Watson,Southampton England,Southampton England,36.0,84.0,464,Nick Catone,"Englewood, NJ USA","Brick, NJ USA",37.0,84.0,red,DEC


In [106]:
new_ufc_fighters_df .to_csv("Data/New_UFC_FightersDF.csv", index=False, header=True)
new_ranking_df.to_csv("Data/New_RankDF.csv", index=False, header=True)

In [107]:
# Fighters Table
new_ufc_fighters_df.head()



,Event_ID,Fight_ID,Date,B_ID,B_Name,B_HomeTown,B_Location,B_Age,B_Weight,R_ID,R_Name,R_HomeTown,R_Location,R_Age,R_Weight,WinningColor,Winby
0,646,4580,2/3/14,808,Alistair Overeem,Hounslow England,Amsterdam The Netherlands,38.0,120.0,377,Frank Mir,"Las Vegas, Nevada USA","Las Vegas, Nevada USA",39.0,119.0,blue,DEC
1,646,4589,2/3/14,1054,Ricardo Lamas,"Chicago, Illinois United States","Chicago, Illinois United States",36.0,65.0,1052,Jose Aldo,Manaus Brazil,Rio de Janeiro Brazil,32.0,65.0,red,DEC
2,646,4590,2/3/14,959,Urijah Faber,"Isla Vista , California USA","Sacramento, California USA",39.0,61.0,1527,Renan Barao,Natal Brazil,Rio de Janeiro Brazil,31.0,61.0,red,KO/TKO
3,646,4605,2/3/14,1056,Danny Martinez,"San Diego, CA USA","San Diego, CA USA",33.0,56.0,1253,Chris Cariaso,"San Jose, California USA","Tucson, Arizona USA",37.0,56.0,red,DEC
4,646,4631,2/3/14,2005,Tom Watson,Southampton England,Southampton England,36.0,84.0,464,Nick Catone,"Englewood, NJ USA","Brick, NJ USA",37.0,84.0,red,DEC


In [108]:
# Rankings Table
new_ranking_df.head()

,date,fighter,weightclass,rank
0,2/4/13,Anderson Silva,Pound-for-Pound,1
1,2/4/13,Jon Jones,Pound-for-Pound,2
2,2/4/13,Georges St-Pierre,Pound-for-Pound,3
3,2/4/13,Jose Aldo,Pound-for-Pound,4
4,2/4/13,Benson Henderson,Pound-for-Pound,5


In [109]:
# Fight Table
fighters_df = ufc_fighters_df[[ "Event_ID", "Fight_ID", "Date",'B_Name', 'B_HomeTown', 'B_Age', 'B_Weight',
                                       'R_Name', 'R_HomeTown', 'R_Age', 'R_Weight','winner', 'winby']].copy()


fighters_df.head()

,Event_ID,Fight_ID,Date,B_Name,B_HomeTown,B_Age,B_Weight,R_Name,R_HomeTown,R_Age,R_Weight,winner,winby
0,646,4580,2/3/14,Alistair Overeem,Hounslow England,38.0,120.0,Frank Mir,"Las Vegas, Nevada USA",39.0,119.0,blue,DEC
1,646,4589,2/3/14,Ricardo Lamas,"Chicago, Illinois United States",36.0,65.0,Jose Aldo,Manaus Brazil,32.0,65.0,red,DEC
2,646,4590,2/3/14,Urijah Faber,"Isla Vista , California USA",39.0,61.0,Renan Barao,Natal Brazil,31.0,61.0,red,KO/TKO
3,646,4605,2/3/14,Danny Martinez,"San Diego, CA USA",33.0,56.0,Chris Cariaso,"San Jose, California USA",37.0,56.0,red,DEC
4,646,4631,2/3/14,Tom Watson,Southampton England,36.0,84.0,Nick Catone,"Englewood, NJ USA",37.0,84.0,red,DEC


In [111]:
red_blue_fighters_df = fighters_df[["Event_ID", "B_Name", "R_Name", "winner", "winby", "Fight_ID"]].copy()
red_blue_fighters_df.head()

,Event_ID,B_Name,R_Name,winner,winby,Fight_ID
0,646,Alistair Overeem,Frank Mir,blue,DEC,4580
1,646,Ricardo Lamas,Jose Aldo,red,DEC,4589
2,646,Urijah Faber,Renan Barao,red,KO/TKO,4590
3,646,Danny Martinez,Chris Cariaso,red,DEC,4605
4,646,Tom Watson,Nick Catone,red,DEC,4631


In [96]:
# # Combining tables on date

# merged_fight_df = pd.merge(new_ufc_fighters_df, new_ranking_df, on="Date")
# merged_fight_df.head()



In [112]:
# transfering  ranking_pd and ufc_fighters_pd back to csv 
red_blue_fighters_df.to_csv("Data/red_blue_fighters_df.csv", index=False, header=True)
#ranking_pd.to_csv("Output/ranking_.csv", index =False, header =True)

In [113]:
rds_connection_string = "root:Megastar@01@127.0.0.1/merged_fight_df"
engine = create_engine(f'mysql://{rds_connection_string}')

In [115]:
#engine.table_names()

In [ ]:
red_blue_fighters_df.to_sql(name='Fighters, con=engine, if_exists='append', index=False)

In [116]:
merged_fight_df.to_sql(name='fight_data', con=engine, if_exists='append', index=False)

InternalError: (pymysql.err.InternalError) (1049, "Unknown database 'merged_fight_df'") (Background on this error at: http://sqlalche.me/e/2j85)

In [98]:
pd.read_sql_query('select * from fight_data', con=engine).head()

,Date,B_ID,B_Name,B_HomeTown,B_Location,B_Age,B_Weight,R_ID,R_Name,R_HomeTown,R_Location,R_Age,R_Weight,WinningColor,Winby,Fighter,WeightClass,Rank
0,2/3/14,808,Alistair Overeem,Hounslow England,Amsterdam The Netherlands,38.0,120.0,377,Frank Mir,"Las Vegas, Nevada USA","Las Vegas, Nevada USA",39.0,119.0,blue,DEC,Jon Jones,Pound-for-Pound,1
1,2/3/14,808,Alistair Overeem,Hounslow England,Amsterdam The Netherlands,38.0,120.0,377,Frank Mir,"Las Vegas, Nevada USA","Las Vegas, Nevada USA",39.0,119.0,blue,DEC,Jose Aldo,Pound-for-Pound,2
2,2/3/14,808,Alistair Overeem,Hounslow England,Amsterdam The Netherlands,38.0,120.0,377,Frank Mir,"Las Vegas, Nevada USA","Las Vegas, Nevada USA",39.0,119.0,blue,DEC,Cain Velasquez,Pound-for-Pound,3
3,2/3/14,808,Alistair Overeem,Hounslow England,Amsterdam The Netherlands,38.0,120.0,377,Frank Mir,"Las Vegas, Nevada USA","Las Vegas, Nevada USA",39.0,119.0,blue,DEC,Renan Barao,Pound-for-Pound,4
4,2/3/14,808,Alistair Overeem,Hounslow England,Amsterdam The Netherlands,38.0,120.0,377,Frank Mir,"Las Vegas, Nevada USA","Las Vegas, Nevada USA",39.0,119.0,blue,DEC,Demetrious Johnson,Pound-for-Pound,5
